In [ ]:
!pip3 install -U scipy
!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 37.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481
Receiving objects: 100% (1481/1481), 53.56 MiB | 14.83 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/tortoise-tts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadat

In [ ]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import IPython
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

from tortoise.utils.text import split_and_recombine_text
from time import time
import os
tts = TextToSpeech()

In [ ]:
!pip install pyngrok flask

In [ ]:
# !ngrok authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxx
# Uncomment above line and enetr token in place of xxxxxxxxxxxxxxxx

In [ ]:
import os
import threading

from flask import Flask, request, jsonify
from pyngrok import ngrok
import base64

app = Flask(__name__)
port = 5000

public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

app.config["BASE_URL"] = public_url

In [ ]:
os.makedirs('results', exist_ok=True)

In [ ]:
os.makedirs('tortoise/voices/new_voice', exist_ok=True)

In [ ]:
@app.route("/",methods=["POST"])
def index():

    data = request.get_json()
    prompt = data['prompt']
    audio_data = base64.b64decode(data['audio'])
    folder_name = 'new_voice'
    seed = int(time())
    voice_outpath = 'results/'

    with open('log.txt','a') as f:
        f.write(voice_outpath)
    with open('tortoise/voices/new_voice/voice.wav','wb') as f:
        f.write(audio_data)
    voice_samples, conditioning_latents = load_voice(folder_name)
    text = prompt
    if '|' in text:
        print("Found the '|' character in your text, which I will use as a cue for where to split it up. If this was not"
            "your intent, please remove all '|' characters from the input.")
        texts = text.split('|')
    else:
        texts = split_and_recombine_text(text)
    all_parts = []
    for j, text in enumerate(texts):
        gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                                preset="ultra_fast", k=1, use_deterministic_seed=seed)
        gen = gen.squeeze(0).cpu()
        torchaudio.save(os.path.join(voice_outpath, f'{j}.wav'), gen, 24000)
        all_parts.append(gen)

    output_name = 'out-voice.wav'
    full_audio = torch.cat(all_parts, dim=-1)
    torchaudio.save(os.path.join(voice_outpath, output_name), full_audio, 24000)
    with open(os.path.join(voice_outpath, output_name),'rb') as f:
        sending = f.read()
    res_aud = base64.b64encode(sending).decode()
    return jsonify({'audio':res_aud})

In [ ]:
# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

In [ ]:
# ngrok.kill()